## 1. dataset join 
## 픽업지, 전달지 건물 정보 + raw_data join 

In [9]:
import pandas as pd
import os 

In [10]:
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [11]:
# dataload
pick_floor = pd.read_excel("prj-ML-model-LT_OV30/pick_floor_data.xls")
pick_info = pd.read_excel("prj-ML-model-LT_OV30/address/shop_final.xls")
data = pd.read_excel("prj-ML-model-LT_OV30/raw_data_time.xlsx")
dlvry = pd.read_excel("prj-ML-model-LT_OV30/address/dlvry_final.xls")

In [12]:
print(pick_floor.isna().sum())
print(pick_info.isna().sum())
print(data.isna().sum())
print(dlvry.isna().sum())

shop_no          0
addr             0
floor            0
rgn1_nm          0
rgn2_nm          0
rgn3_nm          0
shop_category    0
latitude         0
longitude        0
PICK             0
dtype: int64
Latitude        0
Longitude       0
Address         0
건물용도명         212
지상층수_max      212
지하층수_min      212
주용도코드명       1484
건물용도명_2       196
dtype: int64
dlvry_id                 0
business_day             0
reg_hour                 0
service_type             0
shop_no                  0
ord_price                0
notiOver                 0
actual_dlvry_distance    0
shop_loc_pnt_lat         0
shop_loc_pnt_lon         0
dlvry_loc_pnt_lat        0
dlvry_loc_pnt_lon        0
추천노출시간_AI                0
추천노출시간_일반                0
추천미발생시간유                 0
추천미발생시간무                 0
배차후취소시간                  0
전체배차시간                   0
notiOver_min_max         0
per_display              0
per_no_recomm            0
dtype: int64
Latitude         0
Longitude        0
Address          0
건물용

In [13]:
print(pick_floor.shape) #2935
print(pick_info.shape) #1534
print(data.shape) #115,823
print(dlvry.shape) #46951

(2935, 10)
(1534, 8)
(115823, 21)
(46951, 9)


In [14]:
# 픽업지 건물정보 join
pick_floor = pd.merge(pick_floor, pick_info[['Latitude', 'Longitude', 'Address', '건물용도명_2']],
                      left_on=['latitude', 'longitude'], right_on=['Latitude', 'Longitude'], how='left')
print(pick_floor.isna().sum())

shop_no             0
addr                0
floor               0
rgn1_nm             0
rgn2_nm             0
rgn3_nm             0
shop_category       0
latitude            0
longitude           0
PICK                0
Latitude          782
Longitude         782
Address           782
건물용도명_2          1091
dtype: int64


In [15]:
# raw_data + 픽업지 정보
data = pd.merge(data, pick_floor[['shop_no', 'floor', 'rgn2_nm', 'rgn3_nm', 'shop_category', '건물용도명_2', 'Address']], on='shop_no', how='left')
data = data.rename(columns={
    'floor': 'pick_floor',
    'rgn2_nm': 'pick_rgn2_nm',
    'rgn3_nm': 'pick_rgn3_nm',
    '건물용도명_2': 'pick_건물용도',
    'shop_category': 'pick_category',
    'Address': 'pick_address'
})

In [16]:
dlvry['Latitude'] = pd.to_numeric(dlvry['Latitude'], errors='coerce')
dlvry['Longitude'] = pd.to_numeric(dlvry['Longitude'], errors='coerce')
data['dlvry_loc_pnt_lat'] = pd.to_numeric(data['dlvry_loc_pnt_lat'], errors='coerce')
data['dlvry_loc_pnt_lon'] = pd.to_numeric(data['dlvry_loc_pnt_lon'], errors='coerce')

dlvry['Latitude'] = dlvry['Latitude'].round(7).astype(str)
dlvry['Longitude'] = dlvry['Longitude'].round(7).astype(str)
data['dlvry_loc_pnt_lat'] = data['dlvry_loc_pnt_lat'].round(7).astype(str)
data['dlvry_loc_pnt_lon'] = data['dlvry_loc_pnt_lon'].round(7).astype(str)


In [17]:
print(dlvry.info())
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46951 entries, 0 to 46950
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   46951 non-null  object 
 1   Longitude  46951 non-null  object 
 2   Address    46951 non-null  object 
 3   건물용도명      33322 non-null  object 
 4   지상층수_max   33322 non-null  float64
 5   지하층수_min   33322 non-null  float64
 6   주용도코드명     9573 non-null   object 
 7   건물용도명_2    36366 non-null  object 
 8   n          46951 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 3.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115823 entries, 0 to 115822
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   dlvry_id               115823 non-null  object        
 1   business_day           115823 non-null  datetime64[ns]
 2   reg_hour               115823

In [18]:
data = pd.merge(data, dlvry[['Latitude', 'Longitude', 'Address', '지상층수_max', '지하층수_min', '건물용도명_2']],
                left_on=['dlvry_loc_pnt_lat', 'dlvry_loc_pnt_lon'], right_on=['Latitude', 'Longitude'], how='left')
data = data.rename(columns={
    'Address': 'dlvry_address',
    '지상층수_max': 'dlvry_지상층수',
    '지하층수_min': 'dlvry_지하층수',
    '건물용도명_2': 'dlvry_건물용도'
})

In [19]:
cols_to_replace_na = ["pick_floor", "pick_rgn2_nm", "pick_rgn3_nm", "pick_category", "pick_건물용도", "pick_address", "dlvry_address", "dlvry_지상층수", "dlvry_지하층수", "dlvry_건물용도"]
for col in cols_to_replace_na:
    data[col] = data[col].where(data[col] != 'NA')

#print(data.isna().sum())

data.to_csv("prj-ML-model-LT_OV30/raw_data_final.csv", encoding="utf-8", index=False)